Imports

In [1]:
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from datasets import load_dataset

import random

import numpy as np

import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
from collections import defaultdict

import numpy as np
import pandas as pd

from concurrent.futures import ProcessPoolExecutor

import time

/Users/jacob/miniconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Download Data

In [2]:
dataset = load_dataset("ms_marco", "v1.1")

In [3]:
start_time_1 = time.perf_counter()

In [4]:

# train_data = pd.DataFrame(dataset['train'])
test_data = pd.DataFrame(dataset['test'])
# validation_data = pd.DataFrame(dataset['validation'])

In [5]:
def unravel_passages(dataframe):
    unraveled_rows = []

    for index, row in dataframe.iterrows():
        # query_id = row['query_id']
        query = row['query']
        # answers = row['answers']
        count = len(row['passages']['passage_text'])
        
        for passage, url in zip(row['passages']['passage_text'], row['passages']['url']):
            unraveled_rows.append({
                # 'query_id': query_id,
                'query': query,
                # 'answers': answers,
                'passage': passage,
                'url': url,
                'count': count,
            })

    return pd.DataFrame(unraveled_rows)

In [6]:
# unravel_train_data = unravel_passages(train_data)
unravel_test_data = unravel_passages(test_data)
# unravel_val_data = unravel_passages(validation_data)

In [7]:
unique_queries = pd.DataFrame(unravel_test_data['query'].unique(), columns=['query'])
unique_passages = pd.DataFrame(unravel_test_data['passage'].unique(), columns=['passage'])

In [8]:
unique_queries['query_id'] = range(len(unique_queries))
unique_passages['passage_id'] = range(len(unique_passages))

query_to_id = {row['query']: row['query_id'] for index, row in unique_queries.iterrows()}
passage_to_id = {row['passage']: row['passage_id'] for index, row in unique_passages.iterrows()}

unravel_test_data['query_id'] = unravel_test_data['query'].map(query_to_id)
unravel_test_data['passage_id'] = unravel_test_data['passage'].map(passage_to_id)

relevant_passages = unravel_test_data.groupby('query_id')['passage_id'].apply(list).reset_index(name='relevant')

In [9]:
all_passages_ids = set(unique_passages['passage_id'])

In [10]:
def sample_irrelevant_optimized(relevant_list, all_passages_ids, irrelevant_cache, num_samples):
    relevant_set = frozenset(relevant_list)  # Convert to frozenset for hashing
    if relevant_set not in irrelevant_cache:
        possible_irrelevant = list(all_passages_ids - relevant_set)
        irrelevant_cache[relevant_set] = possible_irrelevant
    else:
        possible_irrelevant = irrelevant_cache[relevant_set]
    
    return random.sample(possible_irrelevant, min(len(possible_irrelevant), num_samples))

In [11]:
irrelevant_cache = {}  # Initialize cache outside the function for reuse
relevant_passages['irrelevant'] = relevant_passages['relevant'].apply(
    lambda x: sample_irrelevant_optimized(x, all_passages_ids, irrelevant_cache, len(x))
)

In [12]:
relevant_passages

,query_id,relevant,irrelevant
0,0,"[0, 1, 2, 3, 4, 5, 6]","[30209, 27773, 8616, 38313, 56167, 44861, 52335]"
1,1,"[7, 8, 9, 10, 11, 12, 13, 14, 15]","[36996, 34019, 71965, 18223, 66896, 63445, 551..."
2,2,"[16, 17, 18, 19, 20, 21, 22, 23, 24]","[74387, 49243, 732, 12585, 53876, 67219, 61281..."
3,3,"[25, 26, 27, 28, 29, 30, 31, 32, 33]","[54395, 66623, 74378, 67818, 43131, 13452, 694..."
4,4,"[34, 35, 36, 37, 38, 39, 40, 41, 42, 43]","[14372, 49193, 17109, 50271, 39630, 42264, 404..."
...,...,...,...
9645,9645,"[77856, 77857, 77858, 77859, 77860, 77861, 77862]","[22876, 7481, 68605, 26103, 4918, 34289, 41588]"
9646,9646,"[47682, 77863, 77864, 77865, 77866, 77867, 778...","[59227, 73641, 55954, 15871, 86, 4297, 28879, ..."
9647,9647,"[77871, 77872, 77873, 77874, 77875, 77876, 778...","[33281, 71562, 1196, 74911, 37322, 45103, 1143..."
9648,9648,"[77880, 77881, 77882, 77883, 54246, 77884, 778...","[50982, 49385, 32305, 33354, 24871, 11452, 978..."


In [13]:
end_time_1 = time.perf_counter()

In [14]:
import pandas as pd
from datasets import load_dataset
import random
from collections import defaultdict

def unravel_passages(dataframe):
    unraveled_rows = []
    for index, row in dataframe.iterrows():
        query = row['query']
        count = len(row['passages']['passage_text'])
        for passage, url in zip(row['passages']['passage_text'], row['passages']['url']):
            unraveled_rows.append({
                'query': query,
                'passage': passage,
                'url': url,
                'count': count,
            })
    return pd.DataFrame(unraveled_rows)

def prepare_mappings(unraveled_data):
    unique_queries = pd.DataFrame(unraveled_data['query'].unique(), columns=['query'])
    unique_passages = pd.DataFrame(unraveled_data['passage'].unique(), columns=['passage'])
    unique_queries['query_id'] = range(len(unique_queries))
    unique_passages['passage_id'] = range(len(unique_passages))
    return unique_queries, unique_passages

def map_ids(unraveled_data, query_to_id, passage_to_id):
    unraveled_data['query_id'] = unraveled_data['query'].map(query_to_id)
    unraveled_data['passage_id'] = unraveled_data['passage'].map(passage_to_id)
    return unraveled_data

def sample_irrelevant_optimized(relevant_list, all_passages_ids, num_samples):
    relevant_set = frozenset(relevant_list)
    possible_irrelevant = list(all_passages_ids - relevant_set)
    return random.sample(possible_irrelevant, min(len(possible_irrelevant), num_samples))

def create_triplets_dataframe(unraveled_data):
    relevant_passages = unraveled_data.groupby('query_id')['passage_id'].apply(list).reset_index(name='relevant')
    all_passages_ids = set(unraveled_data['passage_id'])
    irrelevant_cache = {}
    relevant_passages['irrelevant'] = relevant_passages['relevant'].apply(
        lambda x: sample_irrelevant_optimized(x, all_passages_ids, len(x))
    )
    return relevant_passages

def process_dataset(dataset_split):
    unraveled_data = unravel_passages(dataset_split)
    unique_queries, unique_passages = prepare_mappings(unraveled_data)
    query_to_id = {row['query']: row['query_id'] for index, row in unique_queries.iterrows()}
    passage_to_id = {row['passage']: row['passage_id'] for index, row in unique_passages.iterrows()}
    unraveled_data = map_ids(unraveled_data, query_to_id, passage_to_id)
    triplets_df = create_triplets_dataframe(unraveled_data)
    return triplets_df, unique_queries, unique_passages

In [15]:
dataset = load_dataset("ms_marco", "v1.1")


In [16]:
start_time_2 = time.perf_counter()

In [17]:
# train_triplets, train_queries, train_passages = process_dataset(pd.DataFrame(dataset['train']))
test_triplets, test_queries, test_passages = process_dataset(pd.DataFrame(dataset['test']))
# validate_triplets, validate_queries, validate_passages = process_dataset(pd.DataFrame(dataset['validation']))

In [18]:
end_time_2 = time.perf_counter()

In [19]:
time_1 = end_time_1 - start_time_1
time_2 = end_time_2 - start_time_2

print(f"Time 1: {time_1}")
print(f"Time 2: {time_2}")

Time 1: 11.422701625000627
Time 2: 10.06078554099804
